In [ ]:
import csv, nltk, pickle, re, time
import pandas as pd
import numpy as np
from io import StringIO
from collections import Counter
from scipy import sparse
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

# need to use once to download nltk (natural language processing library) on your computer.
# nltk.download()

Vu qu'on a une librarie qui nous permet de faire pas mal de choses, on va:
- mettre tout en minuscule
- retirer la ponctuation
- retirer tous les nombres et caractères non alphanumériques
- les tokeniser: donc séparer les mots
- retirer les stop words (and, the, etc)
- stemmer -> avoir que la racine de chaque mots

à partir de là on aura déjà un dataset plus ou moins propre =)

In [ ]:
# This function does the job of extracting tweet data and formatting it to suit our needs
def format_tweets(filename):
    # Read the data file
    with open("twitter-datasets/" + filename, "r", encoding="utf8") as myfile:
        data = myfile.readlines()
        
    # Make a dataframe out of the data
    tweets = pd.DataFrame(data)
    
    # These are stop words that we want to take out from the tweets
    lang_set = nltk.corpus.stopwords.words('english')

    # The replacement instructions for below
    replacements = [
        ("<user>", ''),
        ("<url>", ''),
        ("'", ' '),
        (r'[^\w\s]', ''),
        (r'[\d]', ''),
        (r'(?:^| )\w(?:$| )', ' ')
    ]
        
    # Put everything in lowercase
    tweets[0] = tweets[0].astype(str).str.lower()
    
    # Take out :
    # - usertags
    # - urls
    # - apostrophes
    # - punctuation and non-alphanumerical characters
    # - numbers
    # - single letter words
    for key, value in replacements:
        tweets[0] = tweets[0].str.replace(key, value)
        
    # Replace blocks of 3 or more times the same letter by a single occurence of the letter
    tweets[0] = tweets[0].apply(lambda x: re.sub(r'([a-zA-Z])\1{2,}', r'\1', x))
        
    # Tokenize each tweet
    tweets[0] = tweets[0].str.split()
    
    # Remove the stop words and the "rt"s (retweets)
    for item in [lang_set, 'rt']:
        tweets[0] = tweets[0].apply(lambda tweet: [word for word in tweet if word not in item])
        
    return tweets

In [ ]:
negative_DF = format_tweets("train_neg.txt")
positive_DF = format_tweets("train_pos.txt")
test_DF = format_tweets("test_data.txt")

In [ ]:
negative_DF.head(10)

J'ai remarké que stemmetize n'est pas toujours la meilleure chose à faire et tente du coup aussi de lemmetize. Ca nous rajoute de ce faite deux colonnes, pour qu'on puisse ensuite sélectionner la meilleure à utiliser.

In [ ]:
# This function applies stemming and lemmatizing to the given tweet set
def stem_and_lem(tweets, stemmer, lemmer):
    tweets['stemmed'] = tweets[0].apply(lambda tweet: [stemmer.stem(word) for word in tweet])
    tweets['lemmed'] = tweets[0].apply(lambda tweet: [lemmatizer.lemmatize(word) for word in tweet])
    tweets['both'] = tweets['lemmed'].apply(lambda tweet: [stemmer.stem(word) for word in tweet])
    
    return tweets

In [ ]:
# Generating the stemmer and lemmatizer
stemmer = nltk.stem.snowball.SnowballStemmer('english') 
lemmatizer = nltk.stem.WordNetLemmatizer()

Les lemmetizer ne semble pas amrcher à 100% donc essayons de faire les deux (donc utiliser le stem sur le lemmetizer)

In [ ]:
negative_DF = stem_and_lem(negative_DF, stemmer, lemmatizer)
positive_DF = stem_and_lem(positive_DF, stemmer, lemmatizer)
test_DF = stem_and_lem(test_DF, stemmer, lemmatizer)

In [ ]:
negative_DF.head(10)

In [ ]:
# This function saves the processed tweets to a txt file
def save_tweets(tweets, filename):
    # Put the stemmed and lemmetized tweets back to string form
    data = tweets['both'].apply(lambda x: ' '.join(x))
    
    # Save to file
    with open("twitter-datasets/" + filename, "w", encoding="utf8") as myfile:
        data.to_csv(myfile, index=False)

In [ ]:
save_tweets(negative_DF, "train_neg_proc.txt")
save_tweets(positive_DF, "train_pos_proc.txt")
save_tweets(test_DF, "test_data_proc.txt")

Maintenant, il va falloir faire ce qui est proposé dans le pdf, donc par exemple, compter les mots qui apparaissent le plus dans negatif et positif, et pourquoi pas utiliser ça ensuite dans notre algorithme pour décider si c'est positif ou négatif dans le test. =)

Quelques petites notes: 
- chaque tweet est un liste de mots
- on a quatre colonnes: les mots de base du tweet, les mots mais stemmés (racine du mot), les mots mais lemmés (idem mais se veut plus précis) et un qui fait les deux (lem puis stem)
- je pars du principe que tout est en anglais.

In [ ]:
# Get embeddings and index values
emb = np.load("outputs/embeddings.npy")

vocab_cut = pd.read_csv("outputs/vocab_cut.txt", sep=" ", header=None, quoting=csv.QUOTE_NONE)
index = pd.Series(vocab_cut[vocab_cut.columns[0]].values)

# Create word definition matrix
word_weights = pd.DataFrame(data=emb, index=index)

word_weights.head()

In [ ]:
# This function removes words that are not in the vocabulary from the tweets
def clean_tweets(tweets, vocab):
    clean = tweets.copy().apply(lambda tweet: [word for word in tweet if word in vocab.values])
    return clean

In [ ]:
# Extract just the column we need
neg_DF = negative_DF["lemmed"]
pos_DF = positive_DF["lemmed"]
test_DF = test_DF["lemmed"]

In [ ]:
pos_DF.head()

<h2>Unused</h2>

In [ ]:
# We want a sparse matrix that tells us for each tweet (rows) how many times a given
# word (columns) appears ("bags of words" representation)

# MIGHT NEED TO BE REDEFINED WITH neg_words AND pos_words INSTEAD !!!!!!
def bags_of_words(tweets=None):
    
    # We get the vocabulary
    with open('outputs/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
        
    col = 0
    data, rows, cols = [], [], []
    
    for tweet in tweets:
        # We count word occurences in a tweet
        word_count = Counter(tweet).items()
        
        # If the word is in the vocabulary we add it in the matrix
        for word, count in word_count:
            row = vocab.get(word)
            
            if row:
                data.append(count)
                cols.append(col)
                rows.append(row)
        
        col += 1
        
    # We convert the scipy.sparse matrix to pandas.SparseSeries for ease of use
    return pd.SparseSeries.from_coo(sparse.coo_matrix((data, (rows, cols))))

In [ ]:
bags = bags_of_words(pos_DF)

In [ ]:
bags.head()

<h2>Random Forest Classifier</h2>

In [ ]:
# Sums the embeddings of each word in the given tweet

# THIS METHOD IS NOT OPTIMAL AND WOULD BENEFIT BEING MADE FASTER UNLESS WE DO NOT USE IT IN THE END
def query_weights(tweet):
    w = pd.DataFrame(columns=range(20))
    
    for word in tweet:
        try:
            w = w.append(word_weights.loc[word, :])
        except KeyError:
            pass
        
    return w.sum(axis=0)

In [ ]:
# Build tweet embeddings
neg_dims = neg_DF.copy().apply(query_weights)
pos_dims = pos_DF.copy().apply(query_weights)
test_dims = test_DF.copy().apply(query_weights)

In [ ]:
# Save the embeddings in pkl files
with open('outputs/neg_dims.pkl', 'wb') as f:
    pickle.dump(neg_dims, f, pickle.HIGHEST_PROTOCOL)
    
with open('outputs/pos_dims.pkl', 'wb') as f:
    pickle.dump(pos_dims, f, pickle.HIGHEST_PROTOCOL)
                
with open('outputs/test_dims.pkl', 'wb') as f:
    pickle.dump(test_dims, f, pickle.HIGHEST_PROTOCOL)            

In [ ]:
# Load the embeddings from pkl files
with open('outputs/neg_dims.pkl', 'rb') as f:
    neg_dims = pickle.load(f)
    
with open('outputs/pos_dims.pkl', 'rb') as f:
    pos_dims = pickle.load(f)
    
with open('outputs/test_dims.pkl', 'rb') as f:
    test_dims = pickle.load(f)

In [ ]:
# Generate the matrices for SVM fitting, we just put the positive and negative embeddings together and
# create the appropriate y matrix with 1's and -1's
X = pos_dims.append(neg_dims)
ones = np.ones((pos_dims.shape[0], 1))
y = np.append(ones, -1 * ones)

In [ ]:
# Applies the Random Forest Classifier technique to the data
start = time.time()
clf = RandomForestClassifier(min_samples_leaf=20)
clf.fit(X, y)
end = time.time()
print("Random Forest", end - start, clf.score(X, y))
pred = pd.DataFrame(clf.predict(test_dims))
pred.columns = ["Prediction"]
pred.insert(0, "Id", pred.index + 1)

In [ ]:
pred.head()

In [ ]:
# We save the submission
pred.to_csv("outputs/submission.csv", index=False, float_format="%.0f")

In [ ]:
# Apply cross validation to the data
scores = cross_val_score(clf, X, y, cv=10)
scores

<h2>TF-IDF</h2>

Create the vectorizer. We go with the idea that we do not want the words that appear in less than 5 tweets and in more than 80% of the tweets.

In [ ]:
# create the vectoriser
vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf =True)

We now need to create a corpus. Our train set would both positive and negative, and our test set is, obviously, the unlabeled part.

To do this, we will append both negative and positive DF, then create a matrix of labels for them.

In [ ]:
# put the list of words into a usable format
neg_DF = pd.DataFrame(neg_DF)
pos_DF = pd.DataFrame(pos_DF)
test_DF = pd.DataFrame(test_DF)
neg_DF["lemmed"] = neg_DF.lemmed.apply(' '.join)
pos_DF["lemmed"] = pos_DF.lemmed.apply(' '.join)
test_DF["lemmed"] = test_DF.lemmed.apply(' '.join)

In [ ]:
# we thus know that all the first ones are labeled as -1 and all the others as 1
all_labeled_DF = pd.concat([neg_DF, pos_DF])

In [ ]:
# we create the labels
negs = len(neg_DF.index)
poss = len(pos_DF.index)
labels = np.zeros(negs+poss)
labels[0:negs]=-1
labels[negs:negs+poss]=1 

In [ ]:
train_corpus_tf_idf = vectorizer.fit_transform(all_labeled_DF["lemmed"]) 
test_corpus_tf_idf = vectorizer.transform(test_DF["lemmed"])

In [ ]:
# create both models
model1 = LinearSVC() # SVM
model2 = MultinomialNB()

In [ ]:
# train on the given models
model1.fit(train_corpus_tf_idf,labels)
model2.fit(train_corpus_tf_idf,labels)

In [ ]:
# predictions
result1 = model1.predict(test_corpus_tf_idf)
result2 = model2.predict(test_corpus_tf_idf)

Result1 and result2 are the labels predicted for the tweets we got in the test corpus. This means we probably jsute have to transforme this into a csv as it is shown in the sample submission.

In [ ]:
# Converting it to integer for prediction csv
result1 = [int(x) for x in result1]
result2 = [int(x) for x in result2]

In [ ]:
svm_df = pd.DataFrame(result1)
svm_df['Id'] = svm_df.index + 1
svm_df['Prediction'] = svm_df[0]
svm_df = svm_df[['Id', 'Prediction']]
svm_df.to_csv('svm.csv', index=False)

In [ ]:
bayes_df = pd.DataFrame(result2)
bayes_df['Id'] = bayes_df.index + 1
bayes_df['Prediction'] = bayes_df[0]
bayes_df = bayes_df[['Id', 'Prediction']]
bayes_df.to_csv('bayes.csv', index=False)